# Overview 
This notebook goes the filtering and estimation of crossover statistic reported in Michaelides et al (2021) for various time intervals. Specifically, we:
- Filter to remove outliers
- Temporally subset the data into different 'thaw season' criteria for 2019 and 2020, based on data collection dates, temperature, and snow data
- compile statistics for each time window and the full dataset

# Setup

In [ ]:
import h5py
import numpy as np
import glob
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
#define some functions to 
def get_rgt_cycle(fname):
    return int(fname[31:39])
def time_from_name(fname):
    return int(fname[16:30])

# Filtering

First we'll filter to remove outliers, defined as anyhting outside of two standard deviations from the mean. For the full crossover dataset, we'll also remove some unneeded columns to sreamline things for for further processing in matlab

In [ ]:
xovers = pd.read_csv('xovers_full_period_low_slopes_new.csv')
#filter out values outside of 2 standard deviations from the mean
xovers_f = xovers[(xovers['dh'] < np.mean(xovers['dh']) + 2*np.std(xovers['dh'])) & (xovers['dh'] > np.mean(xovers['dh']) - 2*np.std(xovers['dh']))]
#compare length before/after filtering
print(len(xovers))
print(len(xovers_f))



In [ ]:
#remove unneeded columns
xovers_f.drop(columns = ['x', 'y','source_1', 'source_2', 'n_points(track_1)', 'n_points(track_2)', 'dh_dy_1', 'dh_dy_2',\
                         'track_1_fit', 'track_2_fit'], inplace = True)

In [ ]:
xovers_f.to_csv('xovers_full_period_low_slopes_new_filtered_demo.csv', index=False) #save filtered dataset

In [ ]:
#filter other crossover datasets by removing everything greater than 2 standard deviations from the mean
xovers_40 = pd.read_csv('xovers_40_full_period_low_slopes_new.csv')
xovers_40_addt = xovers_40[((xovers_40['time_1'] -2019)*365 >= 120) & ((xovers_40['time_2'] -2019)*365 >= 120) & \
                     ((xovers_40['time_1'] -2019*365) <= 290) & ((xovers_40['time_2'] -2019)*365 <= 290)]
xovers_40_f = xovers_40[(xovers_40['dh'] <= np.mean(xovers_40['dh']) + 2*np.std(xovers_40['dh']))\
                                 & (xovers_40['dh'] >= np.mean(xovers_40['dh']) - 2*np.std(xovers_40['dh']))]

xovers_60 = pd.read_csv('xovers_60_full_period_low_slopes_new.csv')
xovers_60_addt = xovers_60[((xovers_60['time_1'] -2019)*365 >= 120) & ((xovers_60['time_2'] -2019)*365 >= 120) & \
                     ((xovers_60['time_1'] -2019*365) <= 290) & ((xovers_60['time_2'] -2019)*365 <= 290)]
xovers_60_f = xovers_60[(xovers_60['dh'] <= np.mean(xovers_60['dh']) + 2*np.std(xovers_60['dh']))\
                                 & (xovers_60['dh'] >= np.mean(xovers_60['dh']) - 2*np.std(xovers_60['dh']))]


xovers_80 = pd.read_csv('xovers_80_full_period_low_slopes_new.csv')
xovers_80_addt = xovers_80[((xovers_80['time_1'] -2019)*365 >= 120) & ((xovers_80['time_2'] -2019)*365 >= 120) & \
                     ((xovers_80['time_1'] -2019*365) <= 290) & ((xovers_80['time_2'] -2019)*365 <= 290)]
xovers_80_f = xovers_80[(xovers_80['dh'] <= np.mean(xovers_80['dh']) + 2*np.std(xovers_80['dh']))\
                                 & (xovers_80['dh'] >= np.mean(xovers_80['dh']) - 2*np.std(xovers_80['dh']))]

xovers_100 = pd.read_csv('xovers_100_full_period_low_slopes_new.csv')
xovers_100_addt = xovers_100[((xovers_100['time_1'] -2019)*365 >= 120) & ((xovers_100['time_2'] -2019)*365 >= 120) & \
                     ((xovers_100['time_1'] -2019*365) <= 290) & ((xovers_100['time_2'] -2019)*365 <= 290)]
xovers_100_f = xovers_100[(xovers_100['dh'] <= np.mean(xovers_100['dh']) + 2*np.std(xovers_100['dh']))\
                                 & (xovers_100['dh'] >= np.mean(xovers_100['dh']) - 2*np.std(xovers_100['dh']))]

# Temporal Subsetting and Statistics 
We now perform additional temporal subsetting based on different thresholds for the thaw season, to calculate the values reported in Table 1 of Michaelides et al (2021).

In [ ]:
#InSAR window: March 8th to October 22nd
xovers_insar_2019 = xovers_f[((xovers_f['time_1'] -np.floor(xovers_f['time_1']))*365 >= 67) & ((xovers_f['time_2'] -np.floor(xovers_f['time_2']))*365 >= 67) & \
                     ((xovers_f['time_1'] -np.floor(xovers_f['time_1']))*365 <= 295) & ((xovers_f['time_2'] -np.floor(xovers_f['time_2']))*365 <= 295) & \
                      (np.floor(xovers_f['time_1']) == 2019) & (np.floor(xovers_f['time_2']) == 2019)]
#Period of positive Accumulated Degree Days of Thaw: April 30th to October 17th
xovers_addt_2019 = xovers_f[((xovers_f['time_1'] -np.floor(xovers_f['time_1']))*365 >= 120) & ((xovers_f['time_2'] -np.floor(xovers_f['time_2']))*365 >= 120) & \
                     ((xovers_f['time_1'] -np.floor(xovers_f['time_1']))*365 <= 290) & ((xovers_f['time_2'] -np.floor(xovers_f['time_2']))*365 <= 290) & \
                      (np.floor(xovers_f['time_1']) == 2019) & (np.floor(xovers_f['time_2']) == 2019)]
#Snow-free interval based on MODIS imagery: June 6th to September 19th
xovers_visual_2019 = xovers_f[((xovers_f['time_1'] -np.floor(xovers_f['time_1']))*365 >= 157) & ((xovers_f['time_2'] -np.floor(xovers_f['time_2']))*365 >= 157) & \
                     ((xovers_f['time_1'] -np.floor(xovers_f['time_1']))*365 <= 260) & ((xovers_f['time_2'] -np.floor(xovers_f['time_2']))*365 <= 260) & \
                      (np.floor(xovers_f['time_1']) == 2019) & (np.floor(xovers_f['time_2']) == 2019)]
#Snow-free interval based on MERRA-2: June 1st to August 31st
xovers_merra_2019 = xovers_f[((xovers_f['time_1'] -np.floor(xovers_f['time_1']))*365 >= 152) & ((xovers_f['time_2'] -np.floor(xovers_f['time_2']))*365 >= 152) & \
                     ((xovers_f['time_1'] -np.floor(xovers_f['time_1']))*365 <= 243) & ((xovers_f['time_2'] -np.floor(xovers_f['time_2']))*365 <= 243) & \
                      (np.floor(xovers_f['time_1']) == 2019) & (np.floor(xovers_f['time_2']) == 2019)]



In [ ]:
#repeat for 2020
#InSAR window: March 8th to October 22nd
xovers_insar_2020 = xovers_f[((xovers_f['time_1'] -np.floor(xovers_f['time_1']))*365 >= 68) & ((xovers_f['time_2'] -np.floor(xovers_f['time_2']))*365 >= 68) & \
                     ((xovers_f['time_1'] -np.floor(xovers_f['time_1']))*365 <= 296) & ((xovers_f['time_2'] -np.floor(xovers_f['time_2']))*365 <= 296) & \
                      (np.floor(xovers_f['time_1']) == 2020) & (np.floor(xovers_f['time_2']) == 2020)]
#Period of positive Accumulated Degree Days of Thaw: April 30th to October 17th
xovers_addt_2020 = xovers_f[((xovers_f['time_1'] -np.floor(xovers_f['time_1']))*365 >= 121) & ((xovers_f['time_2'] -np.floor(xovers_f['time_2']))*365 >= 121) & \
                     ((xovers_f['time_1'] -np.floor(xovers_f['time_1']))*365 <= 291) & ((xovers_f['time_2'] -np.floor(xovers_f['time_2']))*365 <= 291) & \
                      (np.floor(xovers_f['time_1']) == 2020) & (np.floor(xovers_f['time_2']) == 2020)]
#Snow-free interval based on MODIS imagery: June 6th to September 19th
xovers_visual_2020 = xovers_f[((xovers_f['time_1'] -np.floor(xovers_f['time_1']))*365 >= 163) & ((xovers_f['time_2'] -np.floor(xovers_f['time_2']))*365 >= 163) & \
                     ((xovers_f['time_1'] -np.floor(xovers_f['time_1']))*365 <= 262) & ((xovers_f['time_2'] -np.floor(xovers_f['time_2']))*365 <= 262) & \
                      (np.floor(xovers_f['time_1']) == 2020) & (np.floor(xovers_f['time_2']) == 2020)]
#Snow-free interval based on MERRA-2: June 1st to August 31st
xovers_merra_2020 = xovers_f[((xovers_f['time_1'] -np.floor(xovers_f['time_1']))*365 >= 153) & ((xovers_f['time_2'] -np.floor(xovers_f['time_2']))*365 >= 153) & \
                     ((xovers_f['time_1'] -np.floor(xovers_f['time_1']))*365 <= 244) & ((xovers_f['time_2'] -np.floor(xovers_f['time_2']))*365 <= 244) & \
                      (np.floor(xovers_f['time_1']) == 2020) & (np.floor(xovers_f['time_2']) == 2020)]

In [ ]:
#calculate statistics for different time intervals

#define date windows
names = ['original', 'DAYMET thaw/refreeze period', 'Snow free imagery', \
         'MERRA low snow']
date_1 = ['3/8/2019', '4/30/2019', '6/06/2019', '6/01/2019']
date_2 = ['10/22/2019', '10/17/2019', '9/17/2019', '8/31/2019']

medians_2019 = [np.median(xovers_insar_2019['dh']*100), np.median(xovers_addt_2019['dh']*100), np.median(xovers_visual_2019['dh']*100),\
           np.median(xovers_merra_2019['dh']*100)] #median dh (cm) for each window
means_2019 = [np.mean(xovers_insar_2019['dh']*100), np.mean(xovers_addt_2019['dh']*100), np.mean(xovers_visual_2019['dh']*100), \
         np.mean(xovers_merra_2019['dh']*100)] #mean dh (cm) for each window
mins_2019 = [np.min(xovers_insar_2019['dh']*100), np.min(xovers_addt_2019['dh']*100), np.min(xovers_visual_2019['dh']*100), \
             np.min(xovers_merra_2019['dh']*100)]#minimum dh (cm) for each window
maxes_2019 = [np.max(xovers_insar_2019['dh']*100), np.max(xovers_addt_2019['dh']*100), np.max(xovers_visual_2019['dh']*100), \
              np.max(xovers_merra_2019['dh']*100)]#maximum dh (cm) for each window
sigmas_2019 = [np.std(xovers_insar_2019['dh']*100), np.std(xovers_addt_2019['dh']*100), np.std(xovers_visual_2019['dh']*100), \
               np.std(xovers_merra_2019['dh']*100)] #standard deviation of dh (cm) for each window
n_total_2019 = [len(xovers_insar_2019), len(xovers_addt_2019), len(xovers_visual_2019), len(xovers_merra_2019)] #number of crossovers in each windwo
n_short_2019 = [len(xovers_insar_2019[xovers_insar_2019['dt (days)'] <= 14]), \
                len(xovers_addt_2019[xovers_addt_2019['dt (days)'] <= 14]), \
           len(xovers_visual_2019[xovers_visual_2019['dt (days)'] <= 14]), \
                len(xovers_merra_2019[xovers_merra_2019['dt (days)'] <= 14]) ] #number of short-period (< 14 days) crossovers in each interval
sigma_short_2019 = [np.std(xovers_insar_2019[xovers_insar_2019['dt (days)'] <= 14]['dh']*100), \
                    np.std(xovers_addt_2019[xovers_addt_2019['dt (days)'] <= 14]['dh']*100), \
              np.std(xovers_visual_2019[xovers_visual_2019['dt (days)'] <= 14]['dh']*100),\
                    np.std(xovers_merra_2019[xovers_merra_2019['dt (days)'] <= 14]['dh']*100)] #standard deviation of short-period crossovers

In [ ]:
#read in normalized accumulated degree days of thaw data, which account for both thaw (increasing ADDT) and refreeze (decreasing ADDT)
temp = pd.read_csv('NADDT_curve.csv')
doy = temp['DOY'].values
naddt = temp['NADDT'].values
temp.head()

In [ ]:
#count the number of crossovers that correspond to positive and negative changes in ADDT for each window for 2019
n_thaw_2019 = []
n_freeze_2019 = []
insar_thaw_count_19 = 0
insar_freeze_count_19 = 0

for i in range(0, len(xovers_insar_2019)):
    #get DOY for each xover
    doy_1 = int(np.floor((xovers_insar_2019['time_1'].values[i] - 2019)*365))
    doy_2 = int(np.floor((xovers_insar_2019['time_2'].values[i] - 2019)*365))
    delta_addt = naddt[doy_2 - 1] - naddt[doy_1 - 1] #get change in naddt (-1 accounts for the 0 indexing vs DOY starting at 1)
    #check if xover corresponds to freeze or thaw (or neither)
    if delta_addt > 0:
        insar_thaw_count_19 = insar_thaw_count_19+1
    elif delta_addt < 0:
        insar_freeze_count_19 = insar_freeze_count_19 + 1
#add counts to list    
n_thaw_2019.append(insar_thaw_count_19)
n_freeze_2019.append(insar_freeze_count_19)

#repeat for other windows
addt_thaw_count_19 = 0
addt_freeze_count_19 = 0
for i in range(0, len(xovers_addt_2019)):
    #get doy for each xover
    doy_1 = int(np.floor((xovers_addt_2019['time_1'].values[i] - 2019)*365))
    #print(doy_1)
    doy_2 = int(np.floor((xovers_addt_2019['time_2'].values[i] - 2019)*365))
    #print(doy_2)
    delta_addt = naddt[doy_2 - 1] - naddt[doy_1 - 1] #get change in naddt (-1 accounts for the 0 indexing vs doy starting at 1)
    #check if xover corresponds to freeze or thaw (or neither)
    if delta_addt > 0:
        addt_thaw_count_19 = addt_thaw_count_19+1
    elif delta_addt < 0:
        addt_freeze_count_19 = addt_freeze_count_19 + 1

n_thaw_2019.append(addt_thaw_count_19)
n_freeze_2019.append(addt_freeze_count_19)

visual_thaw_count_19 = 0
visual_freeze_count_19 = 0
for i in range(0, len(xovers_visual_2019)):
    #get doy for each xover
    doy_1 = int(np.floor((xovers_visual_2019['time_1'].values[i] - 2019)*365))
    #print(doy_1)
    doy_2 = int(np.floor((xovers_visual_2019['time_2'].values[i] - 2019)*365))
    #print(doy_2)
    delta_addt = naddt[doy_2 - 1] - naddt[doy_1 - 1] #get change in naddt (-1 accounts for the 0 indexing vs doy starting at 1)
    #check if xover corresponds to freeze or thaw (or neither)
    if delta_addt > 0:
        visual_thaw_count_19 = visual_thaw_count_19+1
    elif delta_addt < 0:
        visual_freeze_count_19 = visual_freeze_count_19 + 1

n_thaw_2019.append(visual_thaw_count_19)
n_freeze_2019.append(visual_freeze_count_19)

merra_thaw_count_19 = 0
merra_freeze_count_19 = 0
for i in range(0, len(xovers_merra_2019)):
    #get doy for each xover
    doy_1 = int(np.floor((xovers_merra_2019['time_1'].values[i] - 2019)*365))
    #print(doy_1)
    doy_2 = int(np.floor((xovers_merra_2019['time_2'].values[i] - 2019)*365))
    #print(doy_2)
    delta_addt = naddt[doy_2 - 1] - naddt[doy_1 - 1] #get change in naddt (-1 accounts for the 0 indexing vs doy starting at 1)
    #check if xover corresponds to freeze or thaw (or neither)
    if delta_addt > 0:
        merra_thaw_count_19 = merra_thaw_count_19+1
    elif delta_addt < 0:
        merra_freeze_count_19 = merra_freeze_count_19 + 1

n_thaw_2019.append(merra_thaw_count_19)
n_freeze_2019.append(merra_freeze_count_19)

In [ ]:
#put all the data into a dictionary
data_dict_2019 = {'window': names, 'start date': date_1, 'end date': date_2, 'total crossovers':n_total_2019, 'n_thaw':n_thaw_2019, 'n_freeze':n_freeze_2019, 'mean dh (cm)': means_2019, \
            'median dh (cm)':medians_2019, 'min dh (cm)':mins_2019, 'max dh (cm)':maxes_2019, 'standard deviation (cm)':sigmas_2019,\
             'short period crossovers':n_short_2019, 'short-period standard deviation (cm)':sigma_short_2019 }
data_table_2019 = pd.DataFrame(data_dict_2019)

In [ ]:
#take a look at the statistics for 2019
display(data_table_2019)

In [ ]:
#agregate statistics for 2020
names = ['original', 'DAYMET thaw/refreeze period', 'Snow free imagery', \
         'MERRA low snow']
date_1 = ['3/8/2020', '4/30/2020', '6/11/2020', '6/01/2020']
date_2 = ['10/22/2020', '10/17/2020', '9/18/2020', '8/31/2020']
medians_2020 = [np.median(xovers_insar_2020['dh']*100), np.median(xovers_addt_2020['dh']*100), np.median(xovers_visual_2020['dh']*100),\
           np.median(xovers_merra_2020['dh']*100)] #median dh (cm) for each window
means_2020 = [np.mean(xovers_insar_2020['dh']*100), np.mean(xovers_addt_2020['dh']*100), np.mean(xovers_visual_2020['dh']*100), \
         np.mean(xovers_merra_2020['dh']*100)] #mean dh (cm) for each window
mins_2020 = [np.min(xovers_insar_2020['dh']*100), np.min(xovers_addt_2020['dh']*100), np.min(xovers_visual_2020['dh']*100), \
             np.min(xovers_merra_2020['dh']*100)] #minimum dh (cm) for each window
maxes_2020 = [np.max(xovers_insar_2020['dh']*100), np.max(xovers_addt_2020['dh']*100), np.max(xovers_visual_2020['dh']*100), \
              np.max(xovers_merra_2020['dh']*100)] #maximum dh (cm) for each window
sigmas_2020 = [np.std(xovers_insar_2020['dh']*100), np.std(xovers_addt_2020['dh']*100), np.std(xovers_visual_2020['dh']*100),\
               np.std(xovers_merra_2020['dh']*100)] #standard deviation of dh (cm) for each window
n_total_2020 = [len(xovers_insar_2020), len(xovers_addt_2020), len(xovers_visual_2020), len(xovers_merra_2020)] #number of crossovers in each window
n_short_2020 = [len(xovers_insar_2020[xovers_insar_2020['dt (days)'] <= 14]), len(xovers_addt_2020[xovers_addt_2020['dt (days)'] <= 14]), \
           len(xovers_visual_2020[xovers_visual_2020['dt (days)'] <= 14]), len(xovers_merra_2020[xovers_merra_2020['dt (days)'] <= 14]) ] #number of short-period (< 14 days) crossovers in each interval

sigma_short_2020 = [np.std(xovers_insar_2020[xovers_insar_2020['dt (days)'] <= 14]['dh']*100), \
                    np.std(xovers_addt_2020[xovers_addt_2020['dt (days)'] <= 14]['dh']*100), \
              np.std(xovers_visual_2020[xovers_visual_2020['dt (days)'] <= 14]['dh']*100), \
                    np.std(xovers_merra_2020[xovers_merra_2020['dt (days)'] <= 14]['dh']*100)] #standard deviation of short-period crossovers

In [ ]:
#count the number of crossovers that correspond to positive and negative changes in ADDT for each window

n_thaw_2020 = []
n_freeze_2020 = []
insar_thaw_count_20 = 0
insar_freeze_count_20 = 0

for i in range(0, len(xovers_insar_2020)):
    #get doy for each xover
    doy_1 = int(np.floor((xovers_insar_2020['time_1'].values[i] - 2020)*365))
    #print(doy_1)
    doy_2 = int(np.floor((xovers_insar_2020['time_2'].values[i] - 2020)*365))
    #print(doy_2)
    delta_addt = naddt[doy_2 - 1] - naddt[doy_1 - 1] #get change in naddt (-1 accounts for the 0 indexing vs doy starting at 1)
    #check if xover corresponds to freeze or thaw (or neither)
    if delta_addt > 0:
        insar_thaw_count_20 = insar_thaw_count_20+1
    elif delta_addt < 0:
        insar_freeze_count_20 = insar_freeze_count_20 + 1
    
n_thaw_2020.append(insar_thaw_count_20)
n_freeze_2020.append(insar_freeze_count_20)

addt_thaw_count_20 = 0
addt_freeze_count_20 = 0
for i in range(0, len(xovers_addt_2020)):
    #get doy for each xover
    doy_1 = int(np.floor((xovers_addt_2020['time_1'].values[i] - 2020)*365))
    #print(doy_1)
    doy_2 = int(np.floor((xovers_addt_2020['time_2'].values[i] - 2020)*365))
    #print(doy_2)
    delta_addt = naddt[doy_2 - 1] - naddt[doy_1 - 1] #get change in naddt (-1 accounts for the 0 indexing vs doy starting at 1)
    #check if xover corresponds to freeze or thaw (or neither)
    if delta_addt > 0:
        addt_thaw_count_20 = addt_thaw_count_20+1
    elif delta_addt < 0:
        addt_freeze_count_20 = addt_freeze_count_20 + 1

n_thaw_2020.append(addt_thaw_count_20)
n_freeze_2020.append(addt_freeze_count_20)

visual_thaw_count_20 = 0
visual_freeze_count_20 = 0
for i in range(0, len(xovers_visual_2020)):
    #get doy for each xover
    doy_1 = int(np.floor((xovers_visual_2020['time_1'].values[i] - 2020)*365))
    #print(doy_1)
    doy_2 = int(np.floor((xovers_visual_2020['time_2'].values[i] - 2020)*365))
    #print(doy_2)
    delta_addt = naddt[doy_2 - 1] - naddt[doy_1 - 1] #get change in naddt (-1 accounts for the 0 indexing vs doy starting at 1)
    #check if xover corresponds to freeze or thaw (or neither)
    if delta_addt > 0:
        visual_thaw_count_20 = visual_thaw_count_20+1
    elif delta_addt < 0:
        visual_freeze_count_20 = visual_freeze_count_20 + 1

n_thaw_2020.append(visual_thaw_count_20)
n_freeze_2020.append(visual_freeze_count_20)

merra_thaw_count_20 = 0
merra_freeze_count_20 = 0
for i in range(0, len(xovers_merra_2020)):
    #get doy for each xover
    doy_1 = int(np.floor((xovers_merra_2020['time_1'].values[i] - 2020)*365))
    #print(doy_1)
    doy_2 = int(np.floor((xovers_merra_2020['time_2'].values[i] - 2020)*365))
    #print(doy_2)
    delta_addt = naddt[doy_2 - 1] - naddt[doy_1 - 1] #get change in naddt (-1 accounts for the 0 indexing vs doy starting at 1)
    #check if xover corresponds to freeze or thaw (or neither)
    if delta_addt > 0:
        merra_thaw_count_20 = merra_thaw_count_20+1
    elif delta_addt < 0:
        merra_freeze_count_20 = merra_freeze_count_20 + 1

n_thaw_2020.append(merra_thaw_count_20)
n_freeze_2020.append(merra_freeze_count_20)

In [ ]:
#compile 2020 data into dataframe
data_dict_2020 = {'window': names, 'start date': date_1, 'end date': date_2, 'total crossovers':n_total_2020,'n_thaw':n_thaw_2020, 'n_freeze':n_freeze_2020, 'mean dh (cm)': means_2020, \
            'median dh (cm)':medians_2020, 'min dh (cm)':mins_2020, 'max dh (cm)':maxes_2020, 'standard deviation (cm)':sigmas_2020,\
             'short period crossovers':n_short_2020, 'short-period standard deviation (cm)':sigma_short_2020}
data_table_2020 = pd.DataFrame(data_dict_2020)

In [ ]:
display(data_table_2020)

In [ ]:
#save 2019 and 2020 metrics
data_table_2019.to_csv('xover_time_windows_stats_2019_demo.csv', index = False)
data_table_2020.to_csv('xover_time_windows_stats_2020_demo.csv', index = False)

# Miscellaneous statistics
This section goes through the calcualtion of the general statistics reported in the crossovers section of the paper

In [ ]:
#tally total number of files
os.chdir('Anaktuvuk_v2_ATL06_reduced_all_slopes')
files = glob.glob('*.h5')
track_list = []
for f in files:
    track_list.append(get_rgt_cycle(f)) #record rgt and cycle
print(len(set(track_list))) #get unique tracks 

os.chdir('..')

In [ ]:
xovers_40_f.to_csv('xovers_40_full_period_low_slopes_filtered.csv')
xovers_60_f.to_csv('xovers_60_full_period_low_slopes_filtered.csv')
xovers_80_f.to_csv('xovers_80_full_period_low_slopes_filtered.csv')
xovers_100_f.to_csv('xovers_100_full_period_low_slopes_filtered.csv')

In [ ]:
#get number of crossovers post-filtering for different radii
print(len(xovers_f))
print(len(xovers_40_f))
print(len(xovers_60_f))
print(len(xovers_80_f))
print(len(xovers_100_f))

In [ ]:
#get number of short-period crossovers post-filtering for different radii
print(len(xovers_f[xovers_f['dt (days)'] <= 14]))
print(len(xovers_40_f[xovers_40_f['dt (days)'] <= 14]))
print(len(xovers_60_f[xovers_60_f['dt (days)'] <= 14]))
print(len(xovers_80_f[xovers_80_f['dt (days)'] <= 14]))
print(len(xovers_100_f[xovers_100_f['dt (days)'] <= 14]))

In [ ]:
#check minimum time interval for each file
print(np.min(xovers_f[xovers_f['dt (days)'] > 1]['dt (days)'].values))
print(np.min(xovers_40_f[xovers_40_f['dt (days)'] > 1]['dt (days)'].values))
print(np.min(xovers_60_f[xovers_60_f['dt (days)'] > 1]['dt (days)'].values))
print(np.min(xovers_80_f[xovers_80_f['dt (days)'] > 1]['dt (days)'].values))
print(np.min(xovers_100_f[xovers_100_f['dt (days)'] > 1]['dt (days)'].values))

In [ ]:
#check maximum time interval for each file
print(np.max(xovers_f['dt (days)']))
print(np.max(xovers_40_f['dt (days)']))
print(np.max(xovers_60_f['dt (days)']))
print(np.max(xovers_80_f['dt (days)']))
print(np.max(xovers_100_f['dt (days)']))